In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.contrib import rnn

In [ ]:
mnist = input_data.read_data_sets('/tmp/data', one_hot=True)

In [ ]:
"""
input->weights->hidden layer 1(activation function)->weights->hidden layer 2(activation)->weights->output layer

compare output to intended output -> cost or loss function (cross entropy)
optimization function (optimizer) -> reduce the cost (AdamOptimizer, SGD, AdaGrad,...)

backpropogation

feed forword + backprop = epoch (cycle)
"""

In [ ]:
data_train = mnist.train

In [ ]:
im = data_train.images

In [ ]:
im[0].shape

### Computation Graphs

In [ ]:
total_epochs = 3

n_classes = 10
batch_size = 128

rnn_batch_size = 28
n_rbatches = 28
rnn_size = 128


In [ ]:
x = tf.placeholder('float',[None, n_rbatches, rnn_batch_size])
y = tf.placeholder('float')

In [ ]:
def recurrent_neural_network(x):
    
    # Dictionaries for layers
    layer = {'weights': tf.Variable(tf.random.normal([rnn_size, n_classes])), 
             'biases': tf.Variable(tf.random.normal([n_classes]))}
    
    x = tf.transpose(x, [1,0,2])  
    x = tf.reshape(x, [-1, rnn_batch_size])
    x = tf.split(x, n_rbatches, 0)
    
    lstm_cell = rnn.BasicLSTMCell(rnn_size)
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
                        
    out = tf.add(tf.matmul(outputs[-1], layer['weights']), layer['biases'])   
                    
    return out

### Run

In [ ]:
def train_neural_network(x):
    #feed forward prediction
    predictions = recurrent_neural_network(x)
    
    #backpropogation = minimizing(cost=crossentropy)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    #epoch = feed forward + backprop  and loss at each epoch (epoch_loss)  
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    with tf.Session(config=config) as sess:
        sess.run(tf.initialize_all_variables())
        
        for epoch in range(total_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x,epoch_y = mnist.train.next_batch(batch_size)
                epoch_x = epoch_x.reshape(batch_size, n_rbatches, rnn_batch_size)
                
                _,c = sess.run([optimizer, cost], feed_dict={x: epoch_x, y: epoch_y})
                epoch_loss += c
            print('Epoch',epoch,'completed out of',total_epochs, 'loss:',epoch_loss)
            
        correct = tf.equal(tf.argmax(predictions,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, 'float'))
        print('Accuracy:',accuracy.eval({x:mnist.test.images.reshape((-1, n_rbatches, rnn_batch_size)), y:mnist.test.labels}))

In [ ]:
train_neural_network(x)